## Set the repository path

In [1]:
pyCPT_path = '../'

## Import modules

In [2]:
# Add the repository path into python path
import sys
sys.path.append(pyCPT_path)

# Import modules
import pyCPT
import post_process
import numpy as np

## Read the NGES dataset
#### Note: 1) Data file should contain 3 columns: ( depth, Fr, Qt ); 2) Data should start from the 1st row!

In [ ]:
# By default, model selection using BIC is performed
datapath = '../test_data/NGES_data.csv'
cpt1 = pyCPT.CPT(datapath)

Number of data points:  [296]
Fitting model with number of components =  1
Fitting model with number of components =  2
Fitting model with number of components =  3
Fitting model with number of components =  4


In [ ]:
# If explicitly turn off model selection option, no optimal number of cluster will be inferred.
datapath = '../test_data/NGES_data.csv'
cpt2 = pyCPT.CPT(datapath,do_model_selection=False)

## Extract soil segments

In [ ]:
# if no prior information is available
cpt1.segmentation(num_of_iter=200, start_iter=100)

In [ ]:
# if prior information is available
# prior knowledge of the center of each cluster
prior_mu = np.array([[0.33136292, 1.78557301],
       [0.72919072, 1.43888076],
       [0.63045566, 1.04736416],
       [0.46410682, 1.60826738],
       [0.42322467, 2.35724374],
       [0.77750357, 1.68469789],
       [0.36149448, 1.24393027]])
# the standard deviation of the center coordinates
prior_mu_std = [1,1,2,1,2,1,2]
# prior knowledge of the covariance matrix of each cluster
prior_cov = np.array([[[ 0.00357893,  0.00191449],
        [ 0.00191449,  0.00918239]],

       [[ 0.00320906, -0.0018044 ],
        [-0.0018044 ,  0.00400412]],

       [[ 0.00131583,  0.00111102],
        [ 0.00111102,  0.02355203]],

       [[ 0.00573804, -0.00384005],
        [-0.00384005,  0.00647612]],

       [[ 0.03743975, -0.04114002],
        [-0.04114002,  0.04935895]],

       [[ 0.00114044,  0.00023517],
        [ 0.00023517,  0.0260668 ]],

       [[ 0.01394274, -0.02432719],
        [-0.02432719,  0.04800507]]])

In [ ]:
cpt2.segmentation(num_of_iter=200, start_iter=100, prior_mu=prior_mu, prior_mu_std=prior_mu_std, prior_cov=prior_cov)

## Soil interpretation

In [ ]:
cpt1.detect_layers()
cpt2.detect_layers()

## Visualization

In [ ]:
post_process.plot_feature(cpt1.element,xlabel=['Fr','Qt'], scatter=False) # show the CPT data
post_process.plot_image(cpt1.element, cpt1.element.label_map_est) # show the spatial pattern
pyCPT.mixture_plot_robertson_chart(cpt1, aspect=0.8) # show the statistical pattern
pyCPT.plot_layers(cpt1) # show the soil layers

In [ ]:
post_process.plot_feature(cpt2.element,xlabel=['Fr','Qt'], scatter=False) # show the CPT data
post_process.plot_image(cpt2.element, cpt2.element.label_map_est) # show the spatial pattern
pyCPT.mixture_plot_robertson_chart(cpt2, aspect=0.8) # show the statistical pattern
pyCPT.plot_layers(cpt2) # show the soil layers

#### Identified issue: When you test the repository by using the on-line interactive binder version, some noise contour lines could show up in the contour plot on the Roberson chart. This is an identified issue and probabily related to the version of the package "matplotlib" in MyBinder.org. The colone version from the github repository does not have such behavior.